# jpy_datareader データ取得関数ガイド

このドキュメントでは、`jpy_datareader.data`モジュールで提供される便利な関数について説明します。これらの関数は、各Readerクラスを直接使用せずに、簡単にe-Statデータを取得できるショートカット関数です。

## 概要

`jpy_datareader.data`モジュールには以下の関数が含まれています：
- `get_data_estat_statslist()`: 統計表一覧の取得
- `get_data_estat_metainfo()`: メタ情報の取得  
- `get_data_estat_statsdata()`: 統計データの取得

これらの関数は対応するReaderクラスのラッパー関数として機能し、より簡潔なコードでデータを取得できます。

## 事前準備

### 1. 必要なライブラリのインポート

In [ ]:
import os
import pandas as pd
from dotenv import load_dotenv

# jpy_datareaderのデータ取得関数をインポート
from jpy_datareader.data import (
    get_data_estat_statslist,
    get_data_estat_metainfo,
    get_data_estat_statsdata
)

# 日本語表示の設定
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

### 2. APIキーの設定

In [ ]:
# APIキーの読み込み
load_dotenv()
api_key = os.getenv("ESTAT_APP_ID")

if not api_key:
    print("⚠️ APIキーが設定されていません")
    print("以下の環境変数のいずれかを設定してください:")
    print("- ESTAT_APP_ID")
    print("- E_STAT_APPLICATION_ID")
    print("- ESTAT_APPLICATION_ID")
    print("- E_STAT_API_KEY")
    print("- ESTAT_API_KEY")
    
    # 直接設定する場合（テスト用）
    # api_key = "あなたのAPIキーをここに入力"
    
    # APIキーが設定されていない場合は処理を停止
    raise ValueError("APIキーが設定されていません。環境変数を設定してください。")
else:
    print(f"✅ APIキーが正常に読み込まれました: {api_key[:10]}...")

### 3. 環境変数の設定方法

APIキーは以下の方法で設定できます：

#### 方法1: .envファイルを使用

プロジェクトルートに`.env`ファイルを作成：

```bash
# .envファイルの内容
ESTAT_APP_ID=あなたのAPIキー
```

#### 方法2: 環境変数を直接設定

```bash
# Windows (コマンドプロンプト)
set ESTAT_APP_ID=あなたのAPIキー

# Windows (PowerShell)
$env:ESTAT_APP_ID="あなたのAPIキー"

# macOS/Linux
export ESTAT_APP_ID=あなたのAPIキー
```

#### 方法3: Pythonコード内で直接設定

```python
# テスト用（本番環境では推奨されません）
api_key = "あなたのAPIキー"
```

## 関数詳細

### 1. get_data_estat_statslist()

統計表一覧情報を取得する関数です。`StatsListReader`のショートカット関数として機能します。

#### 基本的な使用方法


In [ ]:
# 基本的な統計表一覧取得
print("=== 基本的な統計表一覧取得 ===")
df_statslist = get_data_estat_statslist(
    api_key=api_key,
    limit=10
)
print(f"取得件数: {len(df_statslist)}")
print(f"カラム: {list(df_statslist.columns)}")

if len(df_statslist) > 0:
    print("\n最新の統計表:")
    latest = df_statslist.iloc[0]
    print(f"ID: {latest.get('id', 'N/A')}")
    print(f"タイトル: {latest.get('TITLE', 'N/A')}")
    print(f"統計名: {latest.get('STAT_NAME', 'N/A')}")

#### 条件を指定した検索

In [ ]:
# 国勢調査の統計表を検索
print("\n=== 国勢調査の統計表検索 ===")
df_census = get_data_estat_statslist(
    api_key=api_key,
    statsCode="00200521",  # 国勢調査
    surveyYears=2020,
    limit=20
)
print(f"国勢調査の統計表: {len(df_census)}件")

# キーワード検索
print("\n=== キーワード検索例 ===")
df_keyword = get_data_estat_statslist(
    api_key=api_key,
    searchWord="人口",
    limit=15
)
print(f"'人口'の検索結果: {len(df_keyword)}件")

# 統計分野での絞り込み
print("\n=== 統計分野での絞り込み ===")
df_field = get_data_estat_statslist(
    api_key=api_key,
    statsField="02",  # 人口・世帯分野
    surveyYears="202001-202312",
    limit=10
)
print(f"人口・世帯分野の統計表: {len(df_field)}件")

#### 利用可能なパラメータ

- `api_key`: e-Stat APIキー（必須）
- `surveyYears`: 調査年月（例: 2020, "2020-2023"）
- `openYears`: 公開年月
- `statsField`: 統計分野コード
- `statsCode`: 政府統計コード
- `searchWord`: 検索キーワード
- `limit`: 取得件数上限
- その他`StatsListReader`で利用可能な全パラメータ

### 2. get_data_estat_metainfo()

統計表のメタ情報を取得する関数です。`MetaInfoReader`のショートカット関数として機能します。

#### 基本的な使用方法

In [ ]:
# 統計表IDの取得（前の検索結果から）
if len(df_statslist) > 0:
    sample_stats_id = df_statslist.iloc[0]['id']
    
    print(f"=== メタ情報取得: {sample_stats_id} ===")
    df_meta = get_data_estat_metainfo(
        api_key=api_key,
        statsDataId=sample_stats_id
    )
    print(f"メタ情報項目数: {len(df_meta)}")
    print(f"メタ情報形状: {df_meta.shape}")
    
    if len(df_meta) > 0:
        print("\nメタ情報サンプル:")
        print(df_meta.head())

#### 階層情報を含む取得

In [ ]:
# 階層情報を含むメタ情報の取得
print("\n=== 階層情報付きメタ情報取得 ===")
df_meta_hierarchy = get_data_estat_metainfo(
    api_key=api_key,
    statsDataId=sample_stats_id,
    has_lv_hierarchy=True,  # 階層情報を取得
    prefix_colname_with_classname=True
)
print(f"階層情報付きメタ情報: {df_meta_hierarchy.shape}")

# 英語モードでの取得
print("\n=== 英語モードでのメタ情報取得 ===")
df_meta_en = get_data_estat_metainfo(
    api_key=api_key,
    statsDataId=sample_stats_id,
    lang="E"  # 英語
)
print(f"英語モードメタ情報: {df_meta_en.shape}")

#### 利用可能なパラメータ

- `api_key`: e-Stat APIキー（必須）
- `statsDataId`: 統計表ID（必須）
- `has_lv_hierarchy`: 階層情報取得フラグ（デフォルト: False）
- `prefix_colname_with_classname`: カラム名にクラス名を付与（デフォルト: True）
- `lang`: 言語設定（"J"=日本語, "E"=英語）
- その他`MetaInfoReader`で利用可能な全パラメータ

### 3. get_data_estat_statsdata()

実際の統計データを取得する関数です。`StatsDataReader`のショートカット関数として機能します。

#### 基本的な使用方法

In [ ]:
# 基本的な統計データ取得
statsDataId = "0003410381"  # 国勢調査：年齢（5歳階級），男女別人口及び人口性比 － 都道府県（大正9年～令和2年）
print(f"=== 基本的な統計データ取得: {sample_stats_id} ===")
df_data = get_data_estat_statsdata(
    api_key=api_key,
    statsDataId=statsDataId,
    limit=1000
)
print(f"データ形状: {df_data.shape}")
print(f"カラム数: {len(df_data.columns)}")

if len(df_data) > 0:
    print("\nデータサンプル:")
    print(df_data.head())
    
    # 数値データの基本統計
    value_cols = [col for col in df_data.columns if '値' in col or col == 'value']
    if value_cols:
        value_col = value_cols[0]
        numeric_data = pd.to_numeric(df_data[value_col], errors='coerce')
        print(f"\n{value_col}の基本統計:")
        print(f"  有効データ件数: {numeric_data.notna().sum():,}件")
        print(f"  平均値: {numeric_data.mean():.2f}")
        print(f"  最大値: {numeric_data.max():.2f}")

#### 条件を指定したデータ取得

In [ ]:
# 地域を絞り込んだデータ取得
print("\n=== 地域絞り込みデータ取得 ===")
df_tokyo = get_data_estat_statsdata(
    api_key=api_key,
    statsDataId=statsDataId,
    cdArea="13000",  # 東京都
    limit=5000
)
print(f"東京都データ: {df_tokyo.shape}")

# 時系列を指定したデータ取得
print("\n=== 時系列指定データ取得 ===")
df_recent = get_data_estat_statsdata(
    api_key=api_key, 
    statsDataId=statsDataId,
    cdTimeFrom="2015",  # 2015年以降
    cdTimeTo="2024",    # 2024年まで
    limit=10000
)
print(f"2015-2024年データ: {df_recent.shape}")

# 複数条件での絞り込み
print("\n=== 複数条件絞り込み ===")
df_filtered = get_data_estat_statsdata(
    api_key=api_key,
    statsDataId=statsDataId,
    lvArea=2,          # 都道府県レベル
    cdTimeFrom="2015", # 2015年以降
    limit=20000
)
print(f"複数条件絞り込み: {df_filtered.shape}")

#### 大容量データの取得

In [ ]:
# 大容量データの自動ページネーション
print("\n=== 大容量データ取得 ===")
df_large = get_data_estat_statsdata(
    api_key=api_key,
    statsDataId=statsDataId,
    limit=150000  # 10万件超でも自動的に分割取得
)
print(f"大容量データ: {df_large.shape}")
print("※ 10万件を超える場合、自動的にページネーション処理が実行されます")

#### 利用可能なパラメータ

- `api_key`: e-Stat APIキー（必須）
- `statsDataId`: 統計表ID（必須）
- `limit`: 取得件数上限
- 絞り込み条件:
  - `cdArea`, `lvArea`, `cdAreaFrom`, `cdAreaTo`: 地域指定
  - `cdTime`, `lvTime`, `cdTimeFrom`, `cdTimeTo`: 時間軸指定
  - `cdTab`, `lvTab`, `cdTabFrom`, `cdTabTo`: 表章項目指定
  - `cdCat01`, `cdCat02`, `cdCat03`: 分類指定
- その他`StatsDataReader`で利用可能な全パラメータ

## 実践的な使用例

### 統合ワークフローの例


In [ ]:
print("=== 統合ワークフロー例 ===")

# ステップ1: 興味のある統計表を検索
print("ステップ1: 統計表検索")
statslist = get_data_estat_statslist(
    api_key=api_key,
    searchWord="人口",
    statsField="02",
    limit=10
)
print(f"見つかった統計表: {len(statslist)}件")

# ステップ2: 最初の統計表のメタ情報を取得
if len(statslist) > 0:
    target_id = statslist.iloc[0]['id']
    target_title = statslist.iloc[0].get('TITLE', '不明')
    
    print(f"\nステップ2: メタ情報取得")
    print(f"対象統計表: {target_title} (ID: {target_id})")
    
    metainfo = get_data_estat_metainfo(
        api_key=api_key,
        statsDataId=target_id,
        has_lv_hierarchy=True
    )
    print(f"メタ情報項目数: {len(metainfo)}")

    # ステップ3: 実際のデータを取得
    print(f"\nステップ3: データ取得")
    
    data = get_data_estat_statsdata(
        api_key=api_key,
        statsDataId=target_id,
        limit=5000
    )
    print(f"取得データ: {data.shape}")
    print(f"主要カラム: {list(data.columns[:5])}")
    
    # ステップ4: 簡単な分析
    print(f"\nステップ4: 基本分析")
    if len(data) > 0:
        # 分類別の件数
        categorical_cols = [col for col in data.columns 
                          if any(keyword in col for keyword in ['地域', '時間', '表章']) 
                          and 'コード' not in col]
        
        for col in categorical_cols[:2]:  # 最初の2つの分類のみ
            if col in data.columns:
                dist = data[col].value_counts()
                print(f"  {col}の分布（上位3項目）:")
                for item, count in dist.head(3).items():
                    print(f"    {item}: {count}件")

### エラーハンドリングの例

In [ ]:

def safe_data_retrieval(api_key, stats_id, max_retries=3):
    """安全なデータ取得の例"""
    
    for attempt in range(max_retries):
        try:
            print(f"データ取得試行 {attempt + 1}/{max_retries}")
            
            # データ取得
            data = get_data_estat_statsdata(
                api_key=api_key,
                statsDataId=stats_id,
                limit=10000
            )
            
            # データ品質チェック
            if len(data) == 0:
                print("⚠️ 空のデータが返されました")
                continue
                
            print(f"✅ データ取得成功: {data.shape}")
            return data
            
        except Exception as e:
            print(f"❌ データ取得エラー (試行 {attempt + 1}): {e}")
            if attempt < max_retries - 1:
                print("リトライします...")
                import time
                time.sleep(2)  # 2秒待機
    
    print("❌ 全ての試行が失敗しました")
    return None

# 使用例
print("\n=== エラーハンドリング例 ===")
if len(statslist) > 0:
    result = safe_data_retrieval(api_key, statslist.iloc[0]['id'])
    if result is not None:
        print(f"安全取得完了: {result.shape}")

### バッチ処理の例

In [ ]:
def batch_data_collection(api_key, search_keywords, max_stats_per_keyword=3):
    """複数キーワードでのバッチデータ収集"""
    
    print("=== バッチデータ収集 ===")
    results = {}
    
    for keyword in search_keywords:
        print(f"\n処理中: '{keyword}'")
        
        try:
            # 統計表検索
            statslist = get_data_estat_statslist(
                api_key=api_key,
                searchWord=keyword,
                limit=max_stats_per_keyword
            )
            print(f"  統計表検索結果: {len(statslist)}件")
            
            # 各統計表のデータを取得
            keyword_data = []
            for i, (_, row) in enumerate(statslist.iterrows()):
                try:
                    print(f"  データ取得 {i+1}/{len(statslist)}: {row['id']}")
                    
                    data = get_data_estat_statsdata(
                        api_key=api_key,
                        statsDataId=row['id'],
                        limit=1000
                    )
                    
                    keyword_data.append({
                        'id': row['id'],
                        'title': row.get('TITLE', ''),
                        'stat_name': row.get('STAT_NAME', ''),
                        'data_shape': data.shape,
                        'data': data
                    })
                    print(f"    ✅ 成功: {data.shape}")
                    
                except Exception as e:
                    print(f"    ❌ エラー: {e}")
                    continue
            
            results[keyword] = keyword_data
            print(f"  '{keyword}'の取得完了: {len(keyword_data)}件")
            
        except Exception as e:
            print(f"  ❌ '{keyword}'の検索でエラー: {e}")
            results[keyword] = []
    
    return results

# 使用例
print("\n=== バッチ処理例 ===")
keywords = ["人口", "労働"]  # 例を2つに制限
batch_results = batch_data_collection(api_key, keywords, max_stats_per_keyword=2)

# 結果の確認
print("\n=== バッチ処理結果 ===")
for keyword, data_list in batch_results.items():
    print(f"'{keyword}': {len(data_list)}件のデータを取得")
    for data_info in data_list:
        print(f"  - {data_info['title']} {data_info['data_shape']}")

## 利点と使い分け

### ショートカット関数の利点

1. **簡潔性**: Readerクラスを直接インスタンス化する必要がない
2. **直感性**: 関数名から機能が明確
3. **効率性**: 一行でデータ取得が可能
4. **学習コストの低減**: 関数ベースでシンプル

### Readerクラスとの使い分け

In [ ]:
# ショートカット関数の使用例（推奨ケース）
print("=== ショートカット関数 vs Readerクラス ===")

print("1. ショートカット関数（簡単なケース）:")
simple_data = get_data_estat_statsdata(
    api_key=api_key,
    statsDataId=sample_stats_id,
    limit=1000
)
print(f"   一行で取得: {simple_data.shape}")

print("\n2. Readerクラス（詳細制御が必要なケース）:")
from jpy_datareader.estat import StatsDataReader

reader = StatsDataReader(
    api_key=api_key,
    statsDataId=sample_stats_id,
    limit=1000
)

# メタデータにアクセス
metadata = reader.get_metadata()
print(f"   統計名: {reader.STAT_NAME}")
print(f"   表題: {reader.TITLE}")

# データ取得
detailed_data = reader.read()
print(f"   データ: {detailed_data.shape}")


**ショートカット関数を使う場合**:
- 単発のデータ取得
- 簡単なスクリプトやプロトタイピング
- 学習・実験段階

**Readerクラスを使う場合**:
- 複雑な処理やカスタマイズが必要
- メタデータ情報への詳細なアクセスが必要
- 複数回のAPI呼び出しを伴う処理
- エラーハンドリングの詳細制御

## まとめ

`jpy_datareader.data`モジュールの関数群を活用することで、e-Statの豊富な統計データに簡単にアクセスし、効率的な分析ワークフローを構築できます。APIキーの適切な設定と、関数の特性を理解した使い分けにより、日本の公的統計データを活用した高度な分析が可能になります。

### 推奨ワークフロー

1. `get_data_estat_statslist()` で興味のある統計表を探す
2. `get_data_estat_metainfo()` でデータ構造を把握
3. `get_data_estat_statsdata()` で実際のデータを取得
4. 取得したデータを分析・可視化

このワークフローにより、日本の公的統計データを効果的に活用した分析が可能になります。

